## =========================================================
## 📄 Digitized Invoices using Gemini | Google AI Studio | JSON
## =========================================================

Load API key from Colab Secrets.
Make sure you've added your API key in the "Secrets" *tab*

In [2]:
from google.colab import userdata
api_key = userdata.get('GOOGLE_API_KEY')

Initialize the Gemini client

In [3]:
from google import genai
client = genai.Client(api_key=api_key)

Upload Invoice *Files*

In [4]:
from google.colab import files
uploaded = files.upload()                       # choose a PDF/JPG/PNG
file_path = next(iter(uploaded))                # first file name
invoice_file = client.files.upload(file=file_path)

Saving invoice.jpeg to invoice (1).jpeg


wait until file is processed

In [5]:
if getattr(invoice_file, "state", "") == "PROCESSING":
    import time
    for _ in range(20):
        time.sleep(0.5)
        f = client.files.get(name=invoice_file.name)
        if getattr(f, "state", "") == "SUCCEEDED":
            invoice_file = f
            break

Import pydantic



In [6]:
from pydantic import BaseModel
from typing import Optional, List

Define schema (Pydantic) for guaranteed JSON shape

In [7]:
class LineItem(BaseModel):
    description: Optional[str]
    quantity: Optional[float]
    unit_price: Optional[float]
    amount: Optional[float]

In [8]:
class Invoice(BaseModel):
    vendor_name: Optional[str]
    invoice_number: Optional[str]
    invoice_date: Optional[str]  # expect YYYY-MM-DD
    due_date: Optional[str]
    currency: Optional[str]
    subtotal: Optional[float]
    tax: Optional[float]
    total: Optional[float]
    line_items: List[LineItem]

Ask for structured JSON that matches the schema

In [9]:
prompt = (
    "Extract fields from this invoice. If a field is absent, set it to null. "
    "Do not invent values. Dates must be YYYY-MM-DD. Return only JSON."
)

In [10]:
res = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[invoice_file, prompt],
    config=genai.types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=Invoice,
        temperature=0.1,
    ),
)

In [11]:
print(res.text)     # JSON string
data = res.parsed

{"vendor_name": "SUNRISE PG FOR MEN", "invoice_number": "600", "invoice_date": "2019-11-19", "due_date": null, "currency": "INR", "subtotal": null, "tax": null, "total": 24000, "line_items": [{"description": "Rent for November 19th 2019 - 28th Nov 2019", "quantity": 1, "unit_price": 24000, "amount": 24000}]}


Save JSON to disk

In [12]:
import json
with open("invoice_output.json", "w") as f:
    json.dump(json.loads(res.text), f, indent=2)

print("Saved to invoice_output.json")

Saved to invoice_output.json
